In [1]:
import pdfplumber
import pandas as pd
import matplotlib.pyplot as plt
import re

plt.style.use('default')

In [2]:
nh_2017_df = pd.read_csv("Budgets/NH/2024 NH State Expenditure.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Budgets/NH/2024 NH State Expenditure.csv'

In [3]:
nh_2017_df = nh_2017_df.dropna(axis=0, how='all').dropna(axis=1, how='all')

NameError: name 'nh_2017_df' is not defined

In [ ]:
nh_2017_df.columns = [col.replace("\n", "").upper for col in nh_2017_df.columns]

In [ ]:
import re

def remove_before_fy(input_string):
    # Find the first occurrence of "FY" followed by exactly two digits
    match = re.search(r'FY\d{2}', input_string)
    if match:
        # Return the string starting after the matched substring
        return input_string[match.end():].strip()
    else:
        # If no match found, return empty string (or original string if preferred)
        return input_string

In [ ]:
re.sub("FY\d{2} ", "", "JUNE FY18 ACTUAL")

In [ ]:
nh_2017_df.describe()

In [ ]:
nh_as_reported_df = pd.DataFrame()

budget_years = range(2016, 2028)

for year in budget_years:
    df = pd.read_csv(f"Budgets/NH/{year} NH State Expenditure.csv")
    df_cleaned = df.dropna(axis=0, how='all').dropna(axis=1, how='all')
    df_cleaned.columns = [col.upper().replace("\n", "").replace("JUNE ", "").replace(r'FY\d{2}', "") for col in df_cleaned.columns]
    df_cleaned.columns = [re.sub("FY\d{2} ", "", col).lstrip() for col in df_cleaned.columns]
    df_appr = df_cleaned[['DEPARTMENT', 'APPROPRIATION']]
    df_appr = df_appr.rename(columns={'APPROPRIATION': year})
    df_appr[year] = df_appr[year].astype(str).str.replace(",", "").astype(float)
    df_appr.set_index('DEPARTMENT', inplace=True)
    nh_as_reported_df = pd.concat([nh_as_reported_df, df_appr], axis=1)

# nh_as_reported_df.set_index(['year', 'DEPARTMENT'], inplace=True)

In [ ]:
nh_as_reported_df.sort_values(by=2025, ascending=False).head(50)

In [ ]:
 #Sort by 2025 descending
sorted_df = nh_as_reported_df.sort_values(by=2025, ascending=False).fillna(0) / 1e9
sorted_df = sorted_df.drop('STATE TOTALS')

# Years
years = list(range(2016, 2026))

# Stackplot
plt.figure(figsize=(12, 8))
plt.stackplot(sorted_df.columns, sorted_df.values)
plt.xlabel('Year')
plt.ylabel('Appropriation')
plt.title('NH State Budget Appropriations by Department (Top 10)')
plt.legend(loc='upper left')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
from 